# CNN Extractor + Regressor

### Constant

In [1]:
DATA_PATH = "../../data.nosync/subj01"
EXTRACTOR = "resnet50-imagenet1k-v2"
LAYER = ["layer1", "layer2", "layer3", "avgpool"]

## Load data

In [2]:
from sklearn.model_selection import train_test_split
from src import dataset

feat, l_frmi, r_frmi = dataset.get_dataset(
    DATA_PATH, EXTRACTOR, LAYER, True)

# normalize
feat = (feat - feat.mean()) / feat.std()

X_train, X_test, l_fmri_train, l_fmri_test, r_fmri_train, r_fmri_test = train_test_split(feat, l_frmi, r_frmi, train_size=0.8)

print("X_train shape: {}".format(X_train.shape))
print("l_fmri_train shape: {}".format(l_fmri_train.shape))
print("r_fmri_train shape: {}".format(r_fmri_train.shape))

print()

print("X_test shape: {}".format(X_test.shape))
print("l_fmri_test shape: {}".format(l_fmri_test.shape))
print("r_fmri_test shape: {}".format(r_fmri_test.shape))

X_train shape: (7872, 3840)
l_fmri_train shape: (7872, 19004)
r_fmri_train shape: (7872, 20544)

X_test shape: (1969, 3840)
l_fmri_test shape: (1969, 19004)
r_fmri_test shape: (1969, 20544)


## Modelling

In [3]:
import numpy as np
from scipy.stats import pearsonr

def compute_perason(pred, target):

    corrcoef = list()
    for pred, target in zip(pred.T, target.T):

        s, _ = pearsonr(x=pred, y=target)
        corrcoef.append(s)

    return np.array(corrcoef)


In [4]:
from sklearn.linear_model import Ridge
# from sklearn.model_selection import cross_validate
# from sklearn.metrics import make_scorer

# model_l = cross_validate(Ridge(alpha=1e4), X=feat, y=l_frmi, cv=5, 
#                          scoring=make_scorer(compute_perason))
# model_l

model_l = Ridge(alpha=5e2).fit(X=X_train, y=l_fmri_train)
y_pred_l = model_l.predict(X_test)

model_r = Ridge(alpha=5e2).fit(X=X_train, y=r_fmri_train)
y_pred_r = model_r.predict(X_test)


In [ ]:
# model_r = cross_validate(Ridge(alpha=1e4), X=feat, y=r_frmi, 
#                          cv=5, scoring=make_scorer(compute_perason))
# model_r


In [5]:
lh_correlation = compute_perason(y_pred_l, l_fmri_test)
rh_correlation = compute_perason(y_pred_r, r_fmri_test)

In [ ]:
import numpy as np

np.save("./ablation_study.nosync/res50_layer3_avgpool_ridge5e3_lh_correlation_dev.npy", lh_correlation)
np.save("./ablation_study.nosync/res50_layer3_avgpool_ridge5e3_rh_correlation_dev.npy", rh_correlation)

## Visualization

### Dev Pearson's R

In [6]:
from src.visualize import histogram, box_plot

histogram(DATA_PATH, lh_correlation, rh_correlation,
          "Ridge, alpha=5e2, {}-{}, Full Image".format(EXTRACTOR, "+".join(LAYER)), "./img/ridge_resnet50_layer1234_img_alpha_5e2/")


/opt/miniconda3/envs/algonauts/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/algonauts/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [10]:
box_plot(DATA_PATH, lh_correlation, rh_correlation,
         "Ridge, alpha=5e2, {}-{}, Full Image".format(EXTRACTOR, "+".join(LAYER)), "./img/ridge_resnet50_layer1234_img_alpha_5e2/")


### Train Pearson's R

In [7]:
y_pred_l_train = model_l.predict(X_train)
y_pred_r_train = model_r.predict(X_train)

In [8]:
lh_correlation_train = compute_perason(y_pred_l_train, l_fmri_train)
rh_correlation_train = compute_perason(y_pred_r_train, r_fmri_train)

In [ ]:
np.save("./ablation_study.nosync/res50_layer3_avgpool_ridge1e5_lh_correlation_train.npy",
        lh_correlation_train)
np.save("./ablation_study.nosync/res50_layer3_avgpool_ridge1e5_rh_correlation_train.npy",
        rh_correlation_train)


In [9]:
histogram(DATA_PATH, lh_correlation_train, rh_correlation_train,
          "Ridge, alpha=5e2, {}-{}, Full Image, Train".format(EXTRACTOR, "+".join(LAYER)), "./img/ridge_resnet50_layer1234_img_alpha_5e2/")


/opt/miniconda3/envs/algonauts/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/opt/miniconda3/envs/algonauts/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



In [11]:
box_plot(DATA_PATH, lh_correlation_train, rh_correlation_train,
         "Ridge, alpha=5e2, {}-{}, Full Image, Train".format(EXTRACTOR, "+".join(LAYER)), "./img/ridge_resnet50_layer1234_img_alpha_5e2/")
